In [ ]:
# MAIN TODO:
#     - put validation set
#     - put optuna
#     - show correct and incorrect predictions
#     - show loss functions decrease through epochs (train, test and val)


# Imports

In [16]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm

from sklearn import metrics

# Helper functions

In [10]:
def get_scores(targets, predictions):
    return {
        "accuracy"         : metrics.accuracy_score(targets, predictions),
        "balanced_accuracy": metrics.balanced_accuracy_score(targets, predictions),
        "f1_score"         : metrics.f1_score(targets, predictions, average="weighted"),
        "precision"        : metrics.precision_score(targets, predictions, average="weighted"),
        "recall"           : metrics.recall_score(targets, predictions, average="weighted"),
        "f1_score"         : metrics.f1_score(targets, predictions, average="weighted")
    }

# Setup

In [11]:
transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                    ])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Def MLP

In [12]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, activation_function):
        super(MLP, self).__init__()
        self.activation_function = activation_function
        self.fc_input   = nn.Linear(in_features=input_size, out_features=64)
        self.fc_hidden1 = nn.Linear(in_features=64, out_features=128)
        self.fc_hidden2 = nn.Linear(in_features=128, out_features=64)
        self.fc_output  = nn.Linear(in_features=64, out_features=num_classes)

    def forward(self, x):
        x = self.activation_function(self.fc_input(x))
        x = self.activation_function(self.fc_hidden1(x))
        x = self.activation_function(self.fc_hidden2(x))
        x = self.fc_output(x)
        return x 

# Def hyperparameters and Loaders

In [13]:
input_size          = 32*32*3
num_classes         = 10

# TODO: let this params manipulated
learning_rate       = 0.001
num_epochs          = 100
batch_size          = 16
activation_function = nn.ReLU()

loss_function       = nn.CrossEntropyLoss()

In [14]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) # to avoid bias
test_loader  = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)

# Main training Loop

In [17]:
mlp = MLP(input_size=input_size, num_classes=num_classes, activation_function=activation_function)
# mlp.cpu()
mlp.cuda()

optimizer = torch.optim.SGD(mlp.parameters(), lr=learning_rate)

best_loss = float('inf')
patience = 5
patience_counter = 0

# main loop
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 32*32*3).cuda()
        labels = labels.cuda()
        optimizer.zero_grad()

        # forward pass
        
        outputs = mlp.forward(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        if (i+1) % 1000 == 0:
            tqdm.write(f' Epoch {epoch + 1}/{num_epochs}, Step {i+1}/{len(train_dataset) // batch_size}, Loss: {loss}')

    epoch_loss /= len(train_loader)
    tqdm.write(f'Epoch {epoch+1} average loss: {epoch_loss:.4f}')

    # early stopping 
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            tqdm.write("Early stopping based on loss reduction through epochs")
            break



  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:01<02:06,  1.28s/it]

Epoch 1 average loss: 2.2991


  2%|▏         | 2/100 [00:02<01:58,  1.21s/it]

Epoch 2 average loss: 2.2928


...

Epoch 98 average loss: 1.0709


 99%|█████████▉| 99/100 [01:55<00:01,  1.07s/it]

Epoch 99 average loss: 1.0610


100%|██████████| 100/100 [01:56<00:00,  1.16s/it]

Epoch 100 average loss: 1.0512


In [19]:
mlp.eval()
predictions = []
labels = []

# running in test set
for images, label in test_loader:
    images = images.view(-1, 32*32*3).cuda()
    label = label.cuda()

    output = mlp.forward(images)
    _, predicted = torch.max(output, 1)

    predictions.extend(predicted.cpu().numpy())
    labels.extend(label.cpu().numpy())

scores = get_scores(labels, predictions)
print(f"Model scores: \n{scores}")



Model scores: 
{'accuracy': 0.6439, 'balanced_accuracy': np.float64(0.6438999999999999), 'f1_score': 0.6410596187680893, 'precision': 0.6434856216467476, 'recall': 0.6439}
